---

## LSTM & variables exogenes

---

## Avantages
1. **Capture des tendances :** Les LSTM mémorisent les variations saisonnières et les tendances à long terme dans les données de ventes.
2. **Gestion des dépendances temporelles :** Adaptés pour modéliser les relations entre les ventes passées et futures.
3. **Précision :** Gèrent les séries temporelles complexes, comme les variations dues à des événements spécifiques (soldes, fêtes).
4. **Flexibilité :** Fonctionnent avec des données multivariées (e.g., météo, promotions) pour des prévisions plus précises.


In [1]:
import pandas as pd
import numpy as np
import plotly_express as px
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from useful_functions import *
file_path = "./results.json"

In [2]:
# --Tables--
series_train = pd.read_csv("series_train.csv")
series_test = pd.read_csv("series_test.csv")

In [3]:
# Inclure les colonnes exogènes
features = ['sales', 'oil_price', 'onpromotion', 'holiday']

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(series_train[features])
scaled_test = scaler.transform(series_test[features])


Pour les réseaux de neurones, il est essentiel de normaliser les données avec `MinMaxScaler` afin de :
- Accélérer la convergence de l'entraînement (pour adam).
- Éviter que des caractéristiques avec des plages de valeurs très différentes n'influencent de manière disproportionnée les poids du modèle.

La mise à l'échelle Min-Max redimensionne les données dans une plage spécifiée, généralement entre 0 et 1, en utilisant la formule suivante :

$$
x_{\text{scaled}} = \frac{x - x_{\min}}{x_{\max} - x_{\min}}
$$

où :
- $x$ : la valeur initiale d'une caractéristique.
- $x_{\min}$ et $x_{\max}$ : les valeurs minimale et maximale de la caractéristique.
- $x_{\text{scaled}}$ : la valeur normalisée, comprise entre 0 et 1.

In [4]:
lag_size = 7 #h=0, on a pas de retard sur les données, 7 trouvé dans le module Sarimax avec analsye acf pacf
X_train, y_train, X_test, y_test = [], [], [], []


# --training--
for i in range(lag_size, len(scaled_train)):
    X_train.append(scaled_train[i - lag_size:i, 0])  # Inclure les séquences de taille `lag_size`
    y_train.append(scaled_train[i, 0])

# --testing--
for i in range(lag_size, len(scaled_test)):
    X_test.append(scaled_test[i - lag_size:i, 0])  # Inclure les séquences de taille `lag_size`
    y_test.append(scaled_test[i, 0])


X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

# Vérifiez les dimensions générées
print("X_train.shape after sequence generation:", X_train.shape)
print("y_train.shape:", y_train.shape)


X_train.shape after sequence generation: (1077, 7)
y_train.shape: (1077,)


 ### Explication du reshape dans les séries temporelles

Lorsqu'on utilise des modèles séquentiels comme les LSTM ou GRU, les données doivent être formatées correctement. Les modèles attendent en entré les données sous la forme suivante :

$$
(n_{\text{samples}}, \text{timesteps}, \text{features})
$$

##### Reshape de `X_test`, $(n_{\text{samples}}, \text{lag\_size})$ --> $(n_{\text{samples}}, \text{timesteps}, \text{features})$

In [5]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [6]:
# initializing the LSTM
regressorLSTM = Sequential()
 
#Adding LSTM layers
regressorLSTM.add(LSTM(units=50,#bcp de neuronne car données pas lineaire, un peu compliqué
                       activation = 'tanh' ,#pour données symetrique c'est bien
                       return_sequences = False, 
                       input_shape = (X_train.shape[1], X_train.shape[2]))) #avec variables exogenes

#Adding the output layer
regressorLSTM.add(Dense(1))
 
#Compiling the model
regressorLSTM.compile(optimizer = 'adam',
                      loss = 'mean_squared_error',
                      metrics = ["accuracy"])
 
#Fitting the model
regressorLSTM.fit(X_train,y_train,
              batch_size=32,
              epochs=50,
              validation_split=0.2
              )
regressorLSTM.summary()

Epoch 1/50
27/27 [==============================] - 1s 14ms/step - loss: 0.0181 - accuracy: 0.0012 - val_loss: 0.0087 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0047 - accuracy: 0.0012 - val_loss: 0.0123 - val_accuracy: 0.0000e+00
Epoch 3/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0045 - accuracy: 0.0012 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 4/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0043 - accuracy: 0.0012 - val_loss: 0.0111 - val_accuracy: 0.0000e+00
Epoch 5/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.0012 - val_loss: 0.0108 - val_accuracy: 0.0000e+00
Epoch 6/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.0012 - val_loss: 0.0107 - val_accuracy: 0.0000e+00
Epoch 7/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.0012 - val_loss: 0.0087 - val_accuracy: 0.

Testing

In [7]:
test_loss = regressorLSTM.evaluate(X_test, y_test)
print("Test Loss:", test_loss)

train_predictions = regressorLSTM.predict(X_train)
test_predictions = regressorLSTM.predict(X_test)

# Inversion des prédictions pour la colonne 'sales'
train_predictions_inversed = scaler.inverse_transform(
    np.concatenate([
        train_predictions, 
        np.zeros((len(train_predictions), scaled_train.shape[1] - 1))  # Colonnes de zéros pour les autres variables
    ], axis=1)
)[:, 0]  # colonne 'sales'

test_predictions_inversed = scaler.inverse_transform(
    np.concatenate([
        test_predictions,
        np.zeros((len(test_predictions), scaled_test.shape[1] - 1))
    ], axis=1)
)[:, 0]


series_train['train_Forecast_LSTM'] = np.nan
series_train.loc[lag_size:, 'train_Forecast_LSTM'] = train_predictions_inversed.flatten()

series_test['test_Forecast_LSTM'] = np.nan
series_test.loc[lag_size:, 'test_Forecast_LSTM'] = test_predictions_inversed.flatten()

19/19 [==============================] - 0s 981us/step - loss: 0.0120 - accuracy: 0.0000e+00
Test Loss: [0.011989344842731953, 0.0]
19/19 [==============================] - 0s 868us/step


In [12]:
fig = px.line(series_train, x='date', y=['sales', 'train_Forecast_LSTM'],title="Predictions du train")
fig.show()
fig = px.line(series_test, x='date', y=['sales', 'test_Forecast_LSTM'],title="Predictions du test")
fig.show()

In [ ]:
nrmse, mape, mae,r2 = calculate_metrics(series_test.loc[lag_size:, 'sales'], series_test.loc[lag_size:, 'test_Forecast_LSTM'])
print(nrmse)
print(mape)
print(mae)
print(r2)

0.07600035162570547
4.968416939537411e+19
142170.38022825713
0.2913158338589046


```python
save_model_results(
    file_path=file_path,
    model_name="LSTM_exogs",
    params={'units' : 50,'activation' : 'tanh'  ,"exogs" : ['oil_price','onpromotion','holiday']},
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)
```

In [11]:
#series_train.to_csv("series_train",index =False, header=True)
#series_test.to_csv("series_test",index =False, header=True)

Les predictions e sont pas meilleurs avec les variables exogenes